<div style="text-align: left; background-color:black; font-family: Trebuchet MS; color:#A60505; padding: 12px; line-height:1.25;border-radius:1px; margin-bottom: 0em; text-align: center; font-size: 58px;border-style: solid;border-color: black solid;"> Threads Reviews EDA + Visualization and Sentiment Analysis </div>

<center>
    <img src='https://imerit.net/wp-content/uploads/2021/07/what-is-sentiment-analysis.jpg'>
</center>

 <a id="toc"></a>
<div class="list-group" id="list-tab" role="tablist"></div>

## <div style="background-color:#B21010;font-family:sans-serif;color:#FFF9ED;font-size:150%;text-align:center;border-radius:9px 9px; padding: 15px; border-style: solid; border-color: black">TABLE OF CONTENTS</div>   
    
* [1. Introduction](#1)
    
* [2. Import Necessaries](#2)
    
* [3. EDA](#3)

    * [3.1. Define data path and dataset name](#3.1)
    * [3.2. Check the shape of the dataset](#3.2)
    * [3.3. Display information about the dataset](#3.3)
    * [3.4. Check the null values in the dataset](#3.4)
    * [3.5. Visualize the Null Values](#3.5)
    * [3.6. Display Unique val in each col](#3.6)
    * [3.7. Extract Insights through visualization](#3.7)
    
    
* [4. Split the data into train, valid and test](#4)
     
* [5. Data Preprocessing](#5)

    * [5.1. TFIDF](#5.1)
    * [5.2. Count Vectorizer](#5.2)
    * [5.3. SVD + Scaling](#5.3)
    * [5.4. Pad Sequences](#5.4)
    * [5.5. Create Embedding Matrix](#5.5)
    * [5.6. one hot encode the target](#5.6)
    

* [6. Machine Learning Modeling](#6)

    * [6.1. Logistic Regression](#6.1)
    
        * [6.1.1. Logistic Regression with TFIDF](#6.1.1)
        * [6.1.2. Logistic Regression with Count Vectroizer](#6.1.2)
        * [6.1.3. Logistic Regression with SVD](#6.1.3)
        
    * [6.2. Naive Bayse](#6.2)
    
        * [6.2.1. Naive Bayse with TFIDF](#6.2.1)
        * [6.2.2. Naive Bayse with Count Vectorizer](#6.2.2)
        
    * [6.3. XGboost](#6.3)
    
        * [6.3.1. XGboos with TFIDF](#6.3.1)
        * [6.3.2. XGboost with Count Vectorizer](#6.3.2)
        * [6.3.3. XGboost with SVD](#6.3.3)
    

* [7. LSTM](#7)

    * [7.1. Model Structure](#7.1)
    * [7.2. Define Early Stop](#7.2)
    * [7.3. Train the model](#7.3)
    

* [8. Evaluate Model](#8)

    * [8.1. Plot accuarcy and loss curve](#8.1)
    * [8.2. Model Accuarcy](#8.2)
    * [8.3. Get Predictions](#8.3)
    * [8.4 Confusion Matrix](#8.4)
    * [8.5 Classification Report](#8.5)
    

* [9. Save the Model](#9)

* [10. Load Model and Predict Inputs](#10)

* [11. AUTHOR MESSAGE](#11)



<a id="1"></a>
## <b>1 <span style='color:#B21010'>||</span> Introduction</b>


<div style="border-radius:10px;
            border: black solid;
            adding: 15px;
            background-color:#FBA7A7;
            font-size:120%;
            text-aling: left">
<h3 style:'border:0; border-radius: 15px; color: black'><center> Steps we will go through:</center></h3>

_________________________________________________________________________
    
Sentiment analysis, a branch of Natural Language Processing (NLP), plays a pivotal role in understanding the pulse of the digital audience. It involves the use of computational techniques to determine the emotional tone, attitude, or subjective information expressed in textual data. Sentiment analysis is not merely about classifying text as "positive," "negative," or "neutral." It is about diving deeper, extracting insights, and discerning the nuanced sentiments that underlie the words.
    
we will walk through this steps:
    
1. Load the data<br>
2. EDA and analyze the data for more insights
3. Split the data into train, test and validation datasets<br>
4. Preprocess the data transform the texts to numerical. a format which computer understand<br>
5. Train some machine learning models on the data such as Logistic Regression, XGboost and Naive Bayse<br>
6. Build LSTM neural network and train the model<br>
7. Evaluate the model <br>
8. Save the model<br>
9. Finally, Load The model and predict inputs
    
after that I will compare the EfficientNet B0 - B7 results

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="2"></a>
## <b>2 <span style='color:#B21010'>||</span>Import Necessaries</b>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.express as px
import missingno as msno
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from tqdm import tqdm

import tensorflow as tf
from keras.utils import np_utils, pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM, GRU, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import load_model


from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="3"></a>
## <b>3 <span style='color:#B21010'>||</span> EDA</b>

<a id="3.1"></a>
### <b>3.1 <span style='color:#B21010'>||</span> Define data path and dataset name</b>

In [ ]:
data_path = '/kaggle/input/threads-app-reviews-dataset/threads_reviews.csv'
ds_name = 'Threads App Reviews'

In [ ]:
df = pd.read_csv(data_path)

In [ ]:
df.head()

<a id="3.2"></a>
### <b>3.2 <span style='color:#B21010'>||</span> Check the shape of the dataset</b>

In [ ]:
def ds_shape(df, ds_name='df'):
    print(f"The {ds_name} dataset has {df.shape[0]} rows and {df.shape[1]}")

ds_shape(df, ds_name)

<a id="3.3"></a>
### <b>3.3 <span style='color:#B21010'>||</span> Display information about the dataset</b>

In [ ]:
def ds_info(df, ds_name='df'):
    print(f"The {ds_name} dataset info:\n")
    print(df.info())
    
ds_info(df, ds_name)

<a id="3.4"></a>
### <b>3.4 <span style='color:#B21010'>||</span> Check the null values in the dataset</b>

In [ ]:
def check_null(df, ds_name='df'):
    print(f"Null Values in each col in the {ds_name} dataset:\n")
    print(df.isnull().sum())
    
check_null(df, ds_name)

<a id="3.5"></a>
### <b>3.5 <span style='color:#B21010'>||</span> Visualize the Null Values</b>

In [ ]:
def plot_miss_vals(df, ds_name="df"):
    msno.matrix(df)
    plt.title(f'Distribution of Missing Values in {ds_name} dataset', fontsize=30, fontstyle='oblique')
    plt.show()
    
plot_miss_vals(df, ds_name)

<a id="3.6"></a>
### <b>3.6 <span style='color:#B21010'>||</span>Display Unique val in each col</b>

In [ ]:
def uni_vals(df, ds_name='df'):
    print(f"The number of unique values in each column in {ds_name} dataset:\n")
    print(df.nunique())

uni_vals(df, ds_name)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="3.7"></a>
### <b>3.7 <span style='color:#B21010'>||</span> Extract Insights through visualization</b>

In [ ]:
df['review_date'] = pd.to_datetime(df['review_date'])
df['review_date'].dtype

In [ ]:
df['review_date_year'] = df['review_date'].dt.strftime('%Y')
df['review_date_month'] = df['review_date'].dt.strftime('%h')
df['review_date_day'] = df['review_date'].dt.strftime('%-d')
df['review_date_dayweek_name'] = df['review_date'].dt.strftime('%a')
df['review_date_hour'] = df['review_date'].dt.strftime('%H')
df['review_date_daytime'] = df['review_date'].dt.strftime('%p')
df.head()

In [ ]:
uni_vals(df, ds_name)

In [ ]:
def plot_top(df, column, title, x_label, y_label, top, template = 'plotly_dark'):
    
    pl = df[column].value_counts().head(top)
    fig = px.bar(y=pl.values, 
                 x=pl.index, 
                 color_discrete_sequence=px.colors.sequential.PuBuGn,
                 text=pl.values,
                 title= title,
                 template= template)
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label,
        font = dict(size=10,family="Franklin Gothic"))
    fig.show()

In [ ]:
def cat_summary_with_graph(dataframe, col_name):
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=('Countplot', 'Percentages'),
                        specs=[[{"type": "xy"}, {'type': 'domain'}]])

    fig.add_trace(go.Bar(y=dataframe[col_name].value_counts().values.tolist(),
                         x=[str(i) for i in dataframe[col_name].value_counts().index],
                         text=dataframe[col_name].value_counts().values.tolist(),
                         textfont=dict(size=15),
                         name=col_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=colors,
                                     line=dict(color='#DBE6EC',
                                               width=1))),
                  row=1, col=1)

    fig.add_trace(go.Pie(labels=dataframe[col_name].value_counts().keys(),
                         values=dataframe[col_name].value_counts().values,
                         textfont=dict(size=20),
                         textposition='auto',
                         showlegend=False,
                         name=col_name,
                         marker=dict(colors=colors)),
                  row=1, col=2)

    fig.update_layout(title={'text': col_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'},
                      template='plotly_white')

    iplot(fig)
    
    
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
              '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']

In [ ]:
df['rating'].value_counts()

In [ ]:
cat_summary_with_graph(df,'rating')

In [ ]:
df['source'].value_counts()

In [ ]:
cat_summary_with_graph(df,'source')

In [ ]:
df['review_date_year'].value_counts()

In [ ]:
df['review_date_month'].value_counts()

**This information confuses me. The threads app was released in July, so how come the app has reviews from January to June? Secondly, it's late August (the date when this notebook was written), so how does it have reviews between September and December, dates that we haven't reached yet? But it doesn't really matter. We're here to finish our job😂**

In [ ]:
plot_top(df, 'review_date_month', "Number of Ratings each month", "Month", "Rating Freq", top=12, template = 'plotly_dark')

In [ ]:
df['review_date_day'].value_counts()

In [ ]:
plot_top(df, 'review_date_day', "Number of Ratings per day", "Day", "Rating Freq", top=21, template = 'plotly_dark')

In [ ]:
df['review_date_dayweek_name'].value_counts()

In [ ]:
cat_summary_with_graph(df,'review_date_dayweek_name')

In [ ]:
df['review_date_hour'].value_counts()

In [ ]:
plot_top(df, 'review_date_hour', "Number of Ratings per hour", "Hour", "Rating Freq", 24, template = 'plotly_dark')

In [ ]:
df['review_date_daytime'].value_counts()

In [ ]:
cat_summary_with_graph(df,'review_date_daytime')

In [ ]:
uni_vals(df)

In [ ]:
google_play = df[df['source'] == 'Google Play']
print(f"Threads App Average Rating on Google Play is {google_play['rating'].mean():.2f} stars")

In [ ]:
app_store = df[df['source'] == 'App Store']
print(f"Threads App Average Rating on App Store is {app_store['rating'].mean():.2f} stars")

In [ ]:
px.histogram(df, x="source", color="rating", title="Ratings Distrbution Per Source")


In [ ]:
px.histogram(df, x="review_date_daytime", color="rating", title="Ratings Distrbution Per Day Time") 

In [ ]:
px.histogram(df, x="review_date_dayweek_name", color="rating", title="Ratings Distrbution Per day week") 

In [ ]:
px.histogram(df, x="review_date_month", color="rating", title = 'Ratings Distrbution Per Month') 

In [ ]:
df['rating'] = df['rating'].astype('int32')

In [ ]:
df['rating'] = df['rating'].replace([1,2], 'bad')
df['rating'] = df['rating'].replace(3, 'neutral')
df['rating'] = df['rating'].replace([4,5], 'good')

In [ ]:
df['rating'].value_counts()

In [ ]:
cat_summary_with_graph(df,'rating')

In [ ]:
X = df['review_description'].values
lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['rating'].values)

y

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="4"></a>
## <b>4 <span style='color:#B21010'>||</span>Split the data into train, valid and test</b>

In [ ]:
X_train, X_dummy, y_train, y_dummy = train_test_split(X, 
                                                      y, 
                                                      stratify=y, 
                                                      random_state=123, 
                                                      test_size=0.3, 
                                                      shuffle=True)

X_valid, X_test, y_valid, y_test = train_test_split(X_dummy, 
                                                    y_dummy, 
                                                    stratify=y_dummy, 
                                                    random_state=123, 
                                                    test_size=0.4, 
                                                    shuffle=True)

In [ ]:
print(f"The training dataset contains {len(X_train)} examples")
print(f"The validation dataset contains {len(X_valid)} examples")
print(f"The testing dataset contains {len(X_test)} examples")

In [ ]:
train_df = pd.DataFrame(
    {
        'Ratings_train' : y_train
    }
)

val_df = pd.DataFrame(
    {
        'Ratings_val' : y_valid
    }
)

test_df = pd.DataFrame(
    {
        'Ratings_test' : y_test
    }
)

In [ ]:
cat_summary_with_graph(train_df,'Ratings_train')

In [ ]:
cat_summary_with_graph(val_df,'Ratings_val')

In [ ]:
cat_summary_with_graph(test_df,'Ratings_test')

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="5"></a>
## <b>5 <span style='color:#B21010'>||</span>Data Preprocessing</b>

<a id="5.1"></a>
### <b>5.1 <span style='color:#B21010'>||</span>TFIDF</b>

In [ ]:
tfv = TfidfVectorizer(min_df=3,  
                      max_features=None, 
                      strip_accents='unicode', 
                      analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), 
                      use_idf=1,
                      smooth_idf=1,
                      sublinear_tf=1,
                      stop_words = 'english')


tfv.fit(list(X_train) + list(X_valid) + list(X_test))
X_train_tfv =  tfv.transform(X_train) 
X_valid_tfv = tfv.transform(X_valid)
X_test_tfv = tfv.transform(X_test)

In [ ]:
def len_data(train, val, test, ds_name='df'):
    print(f"{ds_name} Training dataset has {train.shape[0]} examples and {train.shape[1]} features")
    print(f"{ds_name} Validation dataset has {val.shape[0]} examples and {val.shape[1]} features")
    print(f"{ds_name} Testing dataset has {test.shape[0]} examples and {test.shape[1]} features")
    

print("Using Tfidf:\n")
len_data(X_train_tfv, X_valid_tfv, X_test_tfv, ds_name)

<a id="5.2"></a>
### <b>5.2 <span style='color:#B21010'>||</span>Count Vectorizer</b>

In [ ]:
ctv = CountVectorizer(analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), 
                      stop_words = 'english')

ctv.fit(list(X_train) + list(X_valid) + list(X_test))
X_train_ctv =  ctv.transform(X_train) 
X_valid_ctv = ctv.transform(X_valid)
X_test_ctv = ctv.transform(X_test)

In [ ]:
print("Using Count Vectroizer:\n")

len_data(X_train_ctv, X_valid_ctv, X_test_ctv, ds_name)

<a id="5.3"></a>
### <b>5.3 <span style='color:#B21010'>||</span>SVD + Scaling</b>

In [ ]:
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(X_train_tfv)
X_train_svd = svd.transform(X_train_tfv)
X_valid_svd = svd.transform(X_valid_tfv)
X_test_svd = svd.transform(X_test_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = StandardScaler()
scl.fit(X_train_svd)
X_train_svd_scl = scl.transform(X_train_svd)
X_valid_svd_scl = scl.transform(X_valid_svd)
X_test_svd_scl = scl.transform(X_test_svd)

In [ ]:
print("Using TFIDF SVD Feature:\n")

len_data(X_train_svd, X_valid_svd, X_test_svd, ds_name)

<a id="5.4"></a>
### <b>5.4 <span style='color:#B21010'>||</span>Pad Sequences</b>

In [ ]:
token = text.Tokenizer(num_words=None)
max_len = 500

token.fit_on_texts(list(X_train) + list(X_valid) + list(X_test))
X_train_seq = token.texts_to_sequences(X_train)
X_valid_seq = token.texts_to_sequences(X_valid)
X_test_seq = token.texts_to_sequences(X_test)

# zero pad the sequences
X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_valid_seq = pad_sequences(X_valid_seq, maxlen=max_len)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_len)


word_index = token.word_index

In [ ]:
len_data(X_train_seq, X_valid_seq, X_test_seq, ds_name)

In [ ]:
len(word_index)

<a id="5.5"></a>
### <b>5.5 <span style='color:#B21010'>||</span>Create Embedding Matrix</b>

In [ ]:
embeddings_index = {}
f = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt')
for line in tqdm(f):
    
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        pass
    
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

<a id="5.6"></a>
### <b>5.6 <span style='color:#B21010'>||</span>one hot encode the target</b>

In [ ]:
y_train[:50]

In [ ]:
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_valid)
ytest_enc = np_utils.to_categorical(y_test)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="6"></a>
## <b>6 <span style='color:#B21010'>||</span>Machine Learning Modeling</b>

<a id="6.1"></a>
### <b>6.1 <span style='color:#B21010'>||</span>Logistic Regression</b>

In [ ]:
def logistic_Regression(xtrain, ytrain, xvalid, yvalid, xtest, ytest):
    
    clf = LogisticRegression(C=1.0, max_iter=100000)
    clf.fit(xtrain, ytrain)
    print(f"Accuarcy on Training dataset: {clf.score(xtrain, ytrain)*100:.2f}%")
    print(f"Accuarcy on Validation dataset: {clf.score(xvalid, yvalid)*100:.2f}%")
    print(f"Accuarcy on Testing dataset: {clf.score(xtest, ytest)*100:.2f}%")

<a id="6.1.1"></a>
#### <b>6.1.1 <span style='color:#B21010'>||</span>Logistic Regression with TFIDF</b>

In [ ]:
print("Logistic Regression Results with TFIDF:\n")
logistic_Regression(X_train_tfv, y_train, X_valid_tfv, y_valid, X_test_tfv, y_test)

<a id="6.1.2"></a>
#### <b>6.1.2 <span style='color:#B21010'>||</span>Logistic Regression with Count Vectroizer</b>

In [ ]:
print("Logistic Regression Results with Count Vectroizer:\n")
logistic_Regression(X_train_ctv, y_train, X_valid_ctv, y_valid, X_test_ctv, y_test)

<a id="6.1.3"></a>
#### <b>6.1.3 <span style='color:#B21010'>||</span>Logistic Regression with SVD</b>

In [ ]:
print("Logistic Regression Results with TFIDF SVD Features:\n")
logistic_Regression(X_train_svd_scl, y_train, X_valid_svd_scl, y_valid, X_test_svd_scl, y_test)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="6.2"></a>
### <b>6.2 <span style='color:#B21010'>||</span>Naive Bayse</b>

In [ ]:
def multinomialNB(xtrain, ytrain, xvalid, yvalid, xtest, ytest):
    
    clf = MultinomialNB()
    clf.fit(xtrain, ytrain)
    print(f"Accuarcy on Training dataset: {clf.score(xtrain, ytrain)*100:.2f}%")
    print(f"Accuarcy on Validation dataset: {clf.score(xvalid, yvalid)*100:.2f}%")
    print(f"Accuarcy on Testing dataset: {clf.score(xtest, ytest)*100:.2f}%")

<a id="6.2.1"></a>
#### <b>6.2.1 <span style='color:#B21010'>||</span>Naive Bayse  with TFIDF</b>

In [ ]:
print("multinomialNB Results with TFIDF:\n")
multinomialNB(X_train_tfv, y_train, X_valid_tfv, y_valid, X_test_tfv, y_test)

<a id="6.2.2"></a>
#### <b>6.2.2 <span style='color:#B21010'>||</span>Naive Bayse with count vectroizer</b>

In [ ]:
print("multinomialNB Results with Count Vectroizer:\n")
multinomialNB(X_train_ctv, y_train, X_valid_ctv, y_valid, X_test_ctv, y_test)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="6.3"></a>
### <b>6.3 <span style='color:#B21010'>||</span>XGBoost</b>

In [ ]:
def XGboost(xtrain, ytrain, xvalid, yvalid, xtest, ytest):
    
    clf = xgb.XGBClassifier(max_depth=7, 
                            n_estimators=200, 
                            colsample_bytree=0.8, 
                            subsample=0.8, 
                            nthread=10, 
                            learning_rate=0.1)
    
    clf.fit(xtrain, ytrain)
    print(f"Accuarcy on Training dataset: {clf.score(xtrain, ytrain)*100:.2f}%")
    print(f"Accuarcy on Validation dataset: {clf.score(xvalid, yvalid)*100:.2f}%")
    print(f"Accuarcy on Testing dataset: {clf.score(xtest, ytest)*100:.2f}%")

<a id="6.3.1"></a>
#### <b>6.3.1 <span style='color:#B21010'>||</span>XGboost with TFIDF</b>

In [ ]:
print("XGboost Results with TFIDF:\n")
XGboost(X_train_tfv, y_train, X_valid_tfv, y_valid, X_test_tfv, y_test)

<a id="6.3.2"></a>
#### <b>6.3.2 <span style='color:#B21010'>||</span>Logistic Regression with Count Vectroizer</b>

In [ ]:
print("XGboost Results with Count Vectroizer:\n")
XGboost(X_train_ctv, y_train, X_valid_ctv, y_valid, X_test_ctv, y_test)

<a id="6.3.3"></a>
#### <b>6.3.3 <span style='color:#B21010'>||</span>XGboost with SVD</b>

In [ ]:
print("XGboost Results with TFIDF SVD Feature:\n")
XGboost(X_train_svd, y_train, X_valid_svd, y_valid, X_test_svd, y_test)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="7"></a>
## <b>7 <span style='color:#B21010'>||</span>LSTM</b>

<a id="7.1"></a>
### <b>7.1 <span style='color:#B21010'>||</span>Model Structure</b>

In [ ]:
model = Sequential([
    Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False),
    
    SpatialDropout1D(0.3),
    GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),
    GRU(300, dropout=0.3, recurrent_dropout=0.3),
    
    Dense(1024, activation='relu'),
    Dropout(0.8),
    
    Dense(1024, activation='relu'),
    Dropout(0.8),
    
    Dense(3, activation='softmax')
    
])

model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics= ['accuracy'])

In [ ]:
model.summary()

<a id="7.2"></a>
### <b>7.2 <span style='color:#B21010'>||</span>Define Early Stop</b>

In [ ]:
 early_stopping = EarlyStopping(monitor='val_loss', 
               min_delta=0, 
               patience=5, 
               verbose=0, 
               mode='auto')

<a id="7.3"></a>
#### <b>7.3 <span style='color:#B21010'>||</span>Train The model</b>

In [ ]:
history = model.fit(X_train_seq, 
          y=ytrain_enc, 
          batch_size=512, 
          epochs=100, 
          verbose=1, 
          validation_data=(X_valid_seq, yvalid_enc),
          callbacks=[early_stopping])

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="8"></a>
## <b>8 <span style='color:#B21010'>||</span>Evaluate Model</b>

<a id="8.1"></a>
### <b>8.1 <span style='color:#B21010'>||</span>Plot accuarcy and loss curve </b>

In [ ]:
# Define needed variables
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch={str(index_acc + 1)}'

# Plot training history

plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

<a id="8.2"></a>
### <b>8.2 <span style='color:#B21010'>||</span>Model Accuarcy</b>

In [ ]:
# Evaluate the model on the training dataset
train_scores = model.evaluate(X_train_seq, ytrain_enc, verbose=1)
print(f"Train - Loss: {train_scores[0]:.4f}, Accuracy: {train_scores[1]*100:.2f}%")

# Evaluate the model on the validation dataset
valid_scores = model.evaluate(X_valid_seq, yvalid_enc, verbose=1)
print(f"Validation - Loss: {valid_scores[0]:.4f}, Accuracy: {valid_scores[1]*100:.2f}%")

# Evaluate the model on the test dataset
test_scores = model.evaluate(X_test_seq, ytest_enc, verbose=1)
print(f"Test - Loss: {test_scores[0]:.4f}, Accuracy: {test_scores[1]*100:.2f}%")

<a id="8.3"></a>
### <b>8.3 <span style='color:#B21010'>||</span>Get Prediction</b>

In [ ]:
y_pred = model.predict(X_test_seq)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(ytest_enc, axis=1)

<a id="8.4"></a>
### <b>8.4 <span style='color:#B21010'>||</span>Confussion Matrix</b>

In [ ]:
confusion = confusion_matrix(y_test_classes, y_pred_classes)

class_labels = ['Bad', 'Good', 'Netural']

# Create a confusion matrix heatmap
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)  # Adjust the font size for better readability
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

**The model often gets confused when it comes to the neutral class. In many cases, it mistakenly classifies neutral sentences as either good or bad.
I think next time version I will update the netural comments to be bad comments this will increase the model's accuarcy**

<a id="8.5"></a>
### <b>8.5 <span style='color:#B21010'>||</span>Classification Report</b>

In [ ]:
class_report = classification_report(y_test_classes, y_pred_classes)
print("\nClassification Report:")
print(class_report)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="9"></a>
## <b>9 <span style='color:#B21010'>||</span>Save the Model</b>

In [ ]:
model.save("model.h5")

In [ ]:
with open('word_index.pkl', 'wb') as f:
    pickle.dump(word_index, f)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="10"></a>
## <b>10 <span style='color:#B21010'>||</span>Load the model and Predict the Inputs</b>

In [ ]:
loaded_model = load_model("/kaggle/working/model.h5")

with open('/kaggle/working/word_index.pkl', 'rb') as f:
    word_index = pickle.load(f)
    
tokenizer = text.Tokenizer(num_words=None)
tokenizer.word_index = word_index

max_len = 500 

def preprocess_input(sentence):
    
    sequence = tokenizer.texts_to_sequences([sentence])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    
    return padded_sequence

def predict_sentence_class(sentence):
    input_sequence = preprocess_input(sentence)
    prediction = loaded_model.predict(input_sequence, verbose=0)
    class_labels = ['Bad', 'Good', 'Netural']
    
    predicted_class = class_labels[np.argmax(prediction)]
    
    return predicted_class

def predictive_fun(sen):
    predicted_class = predict_sentence_class(input_sentence)
    print(f"This Review is {predicted_class}")

In [ ]:
input_sentence = "This app is amazing"
predictive_fun(input_sentence)

In [ ]:
input_sentence = "This app is terrible"
predictive_fun(input_sentence)

In [ ]:
input_sentence = "Twitter better than threads"
predictive_fun(input_sentence)

In [ ]:
input_sentence = "Threads better than twitter"
predictive_fun(input_sentence)

In [ ]:
input_sentence = "There is nothing good in this app"
predictive_fun(input_sentence)

<a href="#toc" role="button" aria-pressed="true" >⬆️Back to Table of Contents ⬆️</a>

<a id="11"></a>
## <b>11 <span style='color:#B21010'>||</span>Author Message</b>

<p style="background-color:#DEF5B9;font-family:newtimeroman;color:#1D3E06;font-size:130%;text-align:center;border-radius:20px 20px;"> If you liked this Notebook, please do upvote.<br> If you have any questions, feel free to comment!<br> If you have any advice for me I will be grateful to leave it to me in the comments!<br>✨Best Wishes✨</p>